<a href="https://colab.research.google.com/github/Gopi083/static-template/blob/master/without_imbalance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
df=pd.read_csv('/content/p20.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330452 entries, 0 to 330451
Data columns (total 24 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   330452 non-null  int64  
 1   Date         330452 non-null  object 
 2   User         330452 non-null  object 
 3   L1_norm      330452 non-null  float64
 4   L2_norm      330452 non-null  float64
 5   L3_norm      330452 non-null  float64
 6   L4_norm      330452 non-null  float64
 7   L5_norm      330452 non-null  float64
 8   L6_norm      330452 non-null  float64
 9   L7_norm      330452 non-null  float64
 10  L8_norm      330452 non-null  float64
 11  L9_norm      330452 non-null  float64
 12  E1_norm      330452 non-null  float64
 13  E2_norm      330452 non-null  float64
 14  E3_norm      330452 non-null  float64
 15  E4_norm      330452 non-null  float64
 16  E5_norm      330452 non-null  float64
 17  F1_norm      330452 non-null  float64
 18  H1_norm      330452 non-

In [4]:
del df[df.columns[0]]

In [5]:
y=df['Class_Label']

In [6]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 330452 entries, 0 to 330451
Series name: Class_Label
Non-Null Count   Dtype  
--------------   -----  
330452 non-null  float64
dtypes: float64(1)
memory usage: 2.5 MB


In [7]:
df=df.drop(['Date'],axis=1)

In [8]:
df=df.drop(['User'],axis=1)

In [9]:
X = df.drop('Class_Label', axis=1).values
y = df['Class_Label'].values

In [10]:
df.head()

,L1_norm,L2_norm,L3_norm,L4_norm,L5_norm,L6_norm,L7_norm,L8_norm,L9_norm,E1_norm,...,E3_norm,E4_norm,E5_norm,F1_norm,H1_norm,H2_norm,H3_norm,D1_norm,D2_norm,Class_Label
0,0.884238,0.206870,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.500000,...,0.416667,0.766336,0.500000,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0
1,0.871422,0.402637,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.222222,...,0.666667,0.552669,0.431818,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0
2,0.743272,0.467814,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.555556,...,1.000000,0.703614,0.477273,0.0,0.0,0.035714,0.0,0.0,0.000000,0.0
3,0.704827,0.183623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.388889,...,0.166667,0.637735,0.409091,0.0,0.0,0.000000,0.0,0.0,0.777778,0.0
4,0.704827,0.183623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,...,0.000000,0.708862,0.090909,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [11]:
y = np.nan_to_num(y, nan=-1)

In [12]:
X = np.nan_to_num(X, nan=-1)

In [13]:
print(pd.Series(y).value_counts())

0.0    329486
1.0       966
dtype: int64


In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

# Reshape the data for LSTM input with sequential behavior
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [16]:
from imblearn.under_sampling import RandomUnderSampler

# Undersample the majority class
undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X, y)

In [17]:
scaler = StandardScaler()
X_resampled = scaler.fit_transform(X_resampled)

In [18]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=32, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=32, return_sequences=True))
model.add(LSTM(units=32))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
8262/8262 [==============================] - 68s 8ms/step - loss: 0.0234 - accuracy: 0.9970 - val_loss: 0.0151 - val_accuracy: 0.9973
Epoch 2/20
8262/8262 [==============================] - 63s 8ms/step - loss: 0.0147 - accuracy: 0.9973 - val_loss: 0.0129 - val_accuracy: 0.9973
Epoch 3/20
8262/8262 [==============================] - 61s 7ms/step - loss: 0.0140 - accuracy: 0.9973 - val_loss: 0.0124 - val_accuracy: 0.9973
Epoch 4/20
8262/8262 [==============================] - 62s 8ms/step - loss: 0.0137 - accuracy: 0.9973 - val_loss: 0.0121 - val_accuracy: 0.9973
Epoch 5/20
8262/8262 [==============================] - 66s 8ms/step - loss: 0.0134 - accuracy: 0.9973 - val_loss: 0.0118 - val_accuracy: 0.9973
Epoch 6/20
8262/8262 [==============================] - 70s 8ms/step - loss: 0.0131 - accuracy: 0.9973 - val_loss: 0.0119 - val_accuracy: 0.9973
Epoch 7/20
8262/8262 [==============================] - 64s 8ms/step - loss: 0.0129 - accuracy: 0.9973 - val_loss: 0.0115 - val_ac

In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')

1/1 [==============================] - 2s 2s/step
Accuracy: 1.0
Confusion Matrix:
[[1]]
